# Q-Learning

In [1]:
import numpy as np
import random

In [2]:
R = np.array([[0, 0, 0, -1, 0, 0, -1, 0, 0, 0],
                     [0, -1, -1, -1, 0, 0, 0, 0, -1, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
                     [0, -1, 0, 0, -1, 0, 0, 0, -1, 0],
                     [0, -1, 0, 0, -1, -1, -1, -1, -1, 100],
                     [0, -1, 0, 0, 0, 0, 0, 0, -1, 0],
                     [0, -1, -1, -1, 0, 0, -1, 0, -1, 0],
                     [0, 0, 0, -1, 0, 0, -1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, -1, 0, 0, 0]])

In [3]:
alpha = 0.7
gamma = 0.95
max_epsilon = 1.0
min_epsilon = 0.05
decay_rate = 0.0005
epochs = 10000
actions = ['up', 'down', 'left', 'right']
x = len(R)
y = len(R[0])
states = x*y
Q = np.zeros((x*y, 4))
max_steps = 200
goal_state = 49

In [4]:
state = 0
walls = []

for i in range(x):
    for j in range(y):
        if R[i][j] == -1:
            walls.append(state)
            for a in range(len(actions)):
                Q[state][a] = -1
        state += 1

state = 0

for i in range(x):
    for j in range(y):
        if i == 0 or state - 10 in walls:
            Q[state][actions.index('up')] = -1
        if i == x - 1 or state + 10 in walls:
            Q[state][actions.index('down')] = -1
        if j == 0 or state - 1 in walls:
            Q[state][actions.index('left')] = -1
        if j == y - 1 or state + 1 in walls:
            Q[state][actions.index('right')] = -1
        state += 1

In [5]:
def get_epsilon(epoch):
    return min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * epoch)

In [6]:
def take_action(random_state, epsilon):
    indexes = [index for index,value in enumerate(Q[random_state]) if value != -1]
    possible_actions = [value for index,value in enumerate(actions) if index in indexes]
    if np.random.random() > epsilon:
        if np.max(Q[random_state]) == -1:
            return None
        else:
            return actions[np.argmax(Q[random_state])]
    else:    
        return random.choice(possible_actions)

In [7]:
def update_table(state, action, reward, next_state):
    """
    Update Q table
    """
    Q[state][actions.index(action)] = alpha * (reward + gamma * np.max(Q[next_state]))

In [8]:
%%time
possible_states = [x for x in range(len(Q)) if x not in walls]
for epoch in range(epochs):
    #print(f"Epoch: {epoch}")
    random_state = random.choice(possible_states)
    epsilon = get_epsilon(epoch)
    for steps in range(max_steps):
        action = take_action(random_state, epsilon)
        if action == None:
            break
        elif action == 'up':
            next_state = random_state - 10
        elif action == 'down':
            next_state = random_state + 10
        elif action == 'left':
            next_state = random_state - 1
        elif action == 'right':
            next_state = random_state + 1
        if random_state != goal_state:
            reward = 0
        elif random_state == goal_state:
            reward = 100
        #print(f"Epoch: {epoch}, steps: {steps}, random state: {random_state}, next state: {next_state}")
        update_table(random_state, action, reward, next_state)
        random_state = next_state
        if reward == 100:
            break

CPU times: user 3.14 s, sys: 71.3 ms, total: 3.21 s
Wall time: 3.13 s


In [9]:
def select_state(state):
    action = actions[np.argmax(Q[state])]
    if action == 'up':
        next_state = state - 10
    elif action == 'down':
        next_state = state + 10
    elif action == 'left':
        next_state = state - 1
    elif action == 'right':
        next_state = state + 1
    return next_state

In [10]:
state = random.choice(possible_states)

In [11]:
state

67

In [12]:
path = []

In [13]:
while state != goal_state:
    path.append(state)
    next_state = select_state(state)
    state = next_state
    if state == goal_state:
        path.append(state)
        break

In [14]:
state

49

In [15]:
path

[67, 77, 78, 79, 69, 59, 49]

In [16]:
len(path)

7